<a href="https://colab.research.google.com/github/palit-ishan/Data-Pipeline/blob/main/Data_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Code to scrape data from OpenWeather API using API Key

In [ ]:
import requests
import sqlite3
from datetime import datetime

# API Client
def fetch_weather_data(api_key, city):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Database Storage
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY,
        city TEXT NOT NULL,
        temperature REAL,
        humidity REAL,
        timestamp DATETIME
    );
    '''
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

def insert_data(conn, data):
    sql = ''' INSERT INTO weather_data(city, temperature, humidity, timestamp)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

if __name__ == "__main__":
    api_key = "4d0031a5ae2f0c28e3f8bed5939fabd0"  # Replace with your sample API key
    print('How many cities do you want to enter the data for ?')
    n = input()
    n = int(n)
    for i in range(n):
      k = i+1
      print('City'+ ' No. '+str(k))
      print('Enter US City from where you want the weather data : ')
      city = input()  # Sample city

      # Fetch weather data
      weather_data = fetch_weather_data(api_key, city)

      if weather_data:
          # Extract relevant information
          city_name = weather_data['name']
          temperature = weather_data['main']['temp']
          humidity = weather_data['main']['humidity']
          timestamp = datetime.now()

          # Create SQLite database connection
          database = 'weather_data.db'
          conn = create_connection(database)

          if conn:
              # Create table if not exists
              create_table(conn)

              # Insert data into the database
              data_to_insert = (city_name, temperature, humidity, timestamp)
              insert_data(conn, data_to_insert)

              print("Data successfully inserted into the database.")

              # Close database connection
              conn.close()
          else:
              print("Failed to connect to the database.")
      else:
          print("No weather data fetched. Check your API key or city name.")


How many cities do you want to enter the data for ?
1
City No. 1
Enter US City from where you want the weather data : 
New York
Data successfully inserted into the database.


##Code for Querying Weather Data from Database (SQLite)

In [ ]:
def query_data(conn, query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    return rows

if __name__ == "__main__":
    database = 'weather_data.db'
    conn = sqlite3.connect(database)

    if conn:
        # Example queries
        # Query 1: Retrieve all data
        query1 = "SELECT * FROM weather_data;"
        result1 = query_data(conn, query1)
        print("All weather data:")
        for row in result1:
            print(row)

        # Query 2: Retrieve data for a specific city
        city_name = 'New York'
        query2 = f"SELECT * FROM weather_data WHERE city='{city_name}';"
        result2 = query_data(conn, query2)
        print(f"Weather data for {city_name}:")
        for row in result2:
            print(row)

        # Query 3: Retrieve data for a specific date range (e.g., last 24 hours)
        query3 = "SELECT * FROM weather_data WHERE timestamp >= datetime('now', '-1 day');"
        result3 = query_data(conn, query3)
        print("Weather data for the last 24 hours:")
        for row in result3:
            print(row)

        conn.close()
    else:
        print("Failed to connect to the database.")


All weather data:
(1, 'New York', 278.73, 47.0, '2024-02-03 19:02:50.894937')
(2, 'New York', 278.95, 46.0, '2024-02-03 19:18:55.237756')
(3, 'New York', 278.97, 46.0, '2024-02-03 19:24:18.958230')
(4, 'Boston', 276.03, 50.0, '2024-02-03 19:24:21.979714')
(5, 'Boston', 275.97, 50.0, '2024-02-03 19:25:26.746267')
(6, 'New York', 278.97, 46.0, '2024-02-03 19:25:30.295324')
(7, 'New York', 278.97, 46.0, '2024-02-03 19:26:05.294632')
Weather data for New York:
(1, 'New York', 278.73, 47.0, '2024-02-03 19:02:50.894937')
(2, 'New York', 278.95, 46.0, '2024-02-03 19:18:55.237756')
(3, 'New York', 278.97, 46.0, '2024-02-03 19:24:18.958230')
(6, 'New York', 278.97, 46.0, '2024-02-03 19:25:30.295324')
(7, 'New York', 278.97, 46.0, '2024-02-03 19:26:05.294632')
Weather data for the last 24 hours:
(1, 'New York', 278.73, 47.0, '2024-02-03 19:02:50.894937')
(2, 'New York', 278.95, 46.0, '2024-02-03 19:18:55.237756')
(3, 'New York', 278.97, 46.0, '2024-02-03 19:24:18.958230')
(4, 'Boston', 276.03, 5

## Extending above code to major cities in the US

In [ ]:
import requests
import sqlite3
from datetime import datetime, timedelta
import time

# API Client - Fetch weather data for multiple cities
def fetch_weather_data(api_key, cities):
    weather_data_all_cities = []
    for city in cities:
        url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
        response = requests.get(url)
        if response.status_code == 200:
            weather_data_all_cities.append(response.json())
        else:
            print(f"Failed to fetch data for {city}: {response.status_code}")
    return weather_data_all_cities

# Database Storage - Store weather data in the database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY,
        city TEXT NOT NULL,
        temperature REAL,
        humidity REAL,
        timestamp DATETIME
    );
    '''
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

def insert_data(conn, data):
    sql = ''' INSERT INTO weather_data(city, temperature, humidity, timestamp)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

# Analysis - Perform analysis on the collected data
def analyze_weather_data(conn):
    # Calculate average temperature and humidity for each city
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT city FROM weather_data")
    cities = cursor.fetchall()

    for city in cities:
        city_name = city[0]
        cursor.execute(f"SELECT AVG(temperature), AVG(humidity) FROM weather_data WHERE city='{city_name}'")
        result = cursor.fetchone()
        avg_temperature = result[0]
        avg_humidity = result[1]
        print(f"Average weather in {city_name}:")
        print(f"Average Temperature: {avg_temperature}°C")
        print(f"Average Humidity: {avg_humidity}%")
        print("")

if __name__ == "__main__":
    api_key = '4d0031a5ae2f0c28e3f8bed5939fabd0'
    us_cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio',
                 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'San Francisco', 'Indianapolis',
                 'Columbus', 'Fort Worth', 'Charlotte', 'Seattle', 'Denver', 'El Paso', 'Washington', 'Boston',
                 'Detroit', 'Nashville', 'Portland', 'Memphis', 'Oklahoma City', 'Las Vegas', 'Louisville',
                 'Baltimore', 'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Mesa', 'Sacramento', 'Atlanta',
                 'Kansas City', 'Colorado Springs', 'Miami', 'Raleigh', 'Omaha', 'Long Beach', 'Virginia Beach',
                 'Oakland', 'Minneapolis', 'Tulsa', 'Tampa', 'Arlington']

    database = 'weather_data.db'
    conn = create_connection(database)
    if conn:
        create_table(conn)

        while True:
            weather_data_all_cities = fetch_weather_data(api_key, us_cities)

            if weather_data_all_cities:
                for weather_data in weather_data_all_cities:
                    city_name = weather_data['name']
                    temperature = weather_data['main']['temp']
                    humidity = weather_data['main']['humidity']
                    timestamp = datetime.now()

                    data_to_insert = (city_name, temperature, humidity, timestamp)
                    insert_data(conn, data_to_insert)

                print("Data successfully inserted into the database.")

                # Perform analysis on the collected data
                analyze_weather_data(conn)

            else:
                print("No weather data fetched. Check your API key or city names.")

            # Fetch data every 6 hours
            time.sleep(21600)  # Sleep for 6 hours (21600 seconds)

        conn.close()
    else:
        print("Failed to connect to the database.")